In [39]:
#Импорт файлов и библиотек
import pandas as pd
import numpy as np
import re

#Лемматизация
from pymystem3 import Mystem
m = Mystem()

#исправление опечаток
import jamspell
corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel('/Users/alexanpolyakov/Downloads/ru_small.bin')

True

In [78]:
#Скачиваем все разбитые данные на тексты пользователей и поддержки
data1_clienttexts = pd.read_pickle('data1_clienttexts.pickle')
data1_techtexts = pd.read_pickle('data1_techtexts.pickle')

data2_clienttexts = pd.read_pickle('data2_clienttexts.pickle')
data2_techtexts = pd.read_pickle('data2_techtexts.pickle')

data3_clienttexts = pd.read_pickle('data3_clienttexts.pickle')
data3_techtexts = pd.read_pickle('data3_techtexts.pickle')

In [79]:
data1_clienttexts = data1_clienttexts.reset_index(drop=True)
data1_techtexts = data1_techtexts.reset_index(drop=True)
data2_clienttexts = data2_clienttexts.reset_index(drop=True)
data2_techtexts = data2_techtexts.reset_index(drop=True)
data3_clienttexts = data3_clienttexts.reset_index(drop=True)
data3_techtexts = data3_techtexts.reset_index(drop=True)

In [80]:
#Делаем идентичные имена столбцов
data1_clienttexts = data1_clienttexts.rename(columns={"ID": "id", 'Автор': 'client_id', 'Категория.автора': 'client_category',
                         'Контент': 'text', 'Хост': 'channel_type', 'Подписчики': 'subscribers', 'Тема': 'theme',
                         'Локация': 'location', 'Тональность': 'tonality'})

data1_techtexts = data1_techtexts.rename(columns={'ID': 'id', 'Автор': 'client_id', 'Категория.автора': 'client_category',
                         'Контент': 'text', 'Хост': 'channel_type', 'Подписчики': 'subscribers', 'Тема': 'theme',
                         'Локация': 'location', 'Тональность': 'tonality'})

data2_clienttexts = data2_clienttexts.rename(columns={"appeal_id": "id", 'channel_name': 'location'})
data2_techtexts = data2_techtexts.rename(columns={"appeal_id": "id", 'channel_name': 'location'})

data3_clienttexts = data3_clienttexts.rename(columns={"SENDER_ID": "client_id", "CONTENT": 'text', 'ACCEPTOR': 'location'})
data3_techtexts = data3_techtexts.rename(columns={"SENDER_ID": "client_id", "ANSWERTEXT": 'text', 'ACCEPTOR': 'location'})

In [81]:
#Приводим все данные к одинаковому числу столбцов, отсутствующие столбцы заполняем пустыми значениями
data2_clienttexts = data2_clienttexts.drop('channel_id', axis=1)
data2_techtexts = data2_techtexts.drop('channel_id', axis=1)
data2_clienttexts = data2_clienttexts.assign(client_category = ['nan' for row in range(len(data2_clienttexts))])
data2_techtexts = data2_techtexts.assign(client_category = ['nan' for row in range(len(data2_techtexts))])

data2_clienttexts = data2_clienttexts.assign(theme = ['Прочее' for row in range(len(data2_clienttexts))])
data2_techtexts = data2_techtexts.assign(theme = ['Прочее' for row in range(len(data2_techtexts))])

data2_clienttexts = data2_clienttexts.assign(tonality = ['nan' for row in range(len(data2_clienttexts))])
data2_techtexts = data2_techtexts.assign(tonality = ['nan' for row in range(len(data2_techtexts))])

data2_clienttexts = data2_clienttexts.assign(subscribers = [-1 for row in range(len(data2_clienttexts))])
data2_techtexts = data2_techtexts.assign(subscribers = [-1 for row in range(len(data2_techtexts))])

data3_clienttexts = data3_clienttexts.assign(client_category = ['nan' for row in range(len(data3_clienttexts))])
data3_techtexts = data3_techtexts.assign(client_category = ['nan' for row in range(len(data3_techtexts))])
data3_clienttexts = data3_clienttexts.assign(theme = ['Прочее' for row in range(len(data3_clienttexts))])
data3_techtexts = data3_techtexts.assign(theme = ['Прочее' for row in range(len(data3_techtexts))])

data3_clienttexts = data3_clienttexts.assign(tonality = ['nan' for row in range(len(data3_clienttexts))])
data3_techtexts = data3_techtexts.assign(tonality = ['nan' for row in range(len(data3_techtexts))])

data3_clienttexts = data3_clienttexts.assign(subscribers = [-1 for row in range(len(data3_clienttexts))])
data3_techtexts = data3_techtexts.assign(subscribers = [-1 for row in range(len(data3_techtexts))])

data3_clienttexts = data3_clienttexts.assign(channel_type = ['Почта' for row in range(len(data3_clienttexts))])
data3_techtexts = data3_techtexts.assign(channel_type = ['Почта' for row in range(len(data3_techtexts))])

In [90]:
#Делаем идентичный порядок столбцов
data2_clienttexts = data2_clienttexts[['id', 'client_id', 'client_category', 'text', 'channel_type', 'subscribers',
       'theme', 'location', 'tonality']]
data2_texttexts = data2_clienttexts[['id','client_id', 'client_category', 'text', 'channel_type', 'subscribers',
       'theme', 'location', 'tonality']]

data3_clienttexts = data3_clienttexts[['client_id', 'client_category', 'text', 'channel_type', 'subscribers',
       'theme', 'location', 'tonality']]
data3_texttexts = data3_clienttexts[['client_id', 'client_category', 'text', 'channel_type', 'subscribers',
       'theme', 'location', 'tonality']]

In [91]:
#Создаем полный датасет пользовательских обращений
data_clienttexts = data1_clienttexts
data_clienttexts = data_clienttexts.append(data2_clienttexts)
data_clienttexts = data_clienttexts.append(data3_clienttexts)

In [92]:
#Создаем полный датасет технических ответов
data_techtexts = data1_techtexts
data_techtexts = data_techtexts.append(data2_techtexts)
data_techtexts = data_techtexts.append(data3_techtexts)

In [93]:
data_clienttexts = data_clienttexts.reset_index(drop=True)
data_techtexts = data_techtexts.reset_index(drop=True)

In [98]:
data_clienttexts.to_pickle('data_clienttexts.pickle')
data_techtexts.to_pickle('data_textstexts.pickle')

In [1]:
#data1_clienttexts

In [ ]:
#...